In [2]:
import pandas as pd
import numpy as ap
import lightgbm as lg
import re

In [3]:
DATA_PATH = r"../../../../../Data/Elo Merchant Category Recommendation/"

In [4]:
train = pd.read_csv(DATA_PATH + r"train.csv")
#(201917, 6)
#'first_active_month', 'card_id', 'feature_1', 'feature_2', 'feature_3', 'target'

In [49]:
test = pd.read_csv(DATA_PATH+r"test.csv")
#(123623, 5)
#'first_active_month', 'card_id', 'feature_1', 'feature_2', 'feature_3'

In [5]:
merchants = pd.read_csv(DATA_PATH+r"merchants.csv")
#(334696, 22)
#'merchant_id', 'merchant_group_id', 'merchant_category_id', 'subsector_id', 'numerical_1',
#'numerical_2', 'category_1','most_recent_sales_range', 'most_recent_purchases_range',
#'avg_sales_lag3', 'avg_purchases_lag3', 'active_months_lag3','avg_sales_lag6', 
#'avg_purchases_lag6', 'active_months_lag6','avg_sales_lag12', 'avg_purchases_lag12',
#'active_months_lag12','category_4', 'city_id', 'state_id', 'category_2'

In [6]:
new_merchant = pd.read_csv(DATA_PATH+r"new_merchant_transactions.csv")
#(1963031, 14)
#'authorized_flag', 'card_id', 'city_id', 'category_1', 'installments',
#'category_3', 'merchant_category_id', 'merchant_id', 'month_lag',
#'purchase_amount', 'purchase_date', 'category_2', 'state_id','subsector_id'


In [7]:
hist = pd.read_csv(DATA_PATH+r"historical_transactions.csv")
#(29112361, 14)
#'authorized_flag', 'card_id', 'city_id', 'category_1', 'installments',
#'category_3', 'merchant_category_id', 'merchant_id', 'month_lag',
#'purchase_amount', 'purchase_date', 'category_2', 'state_id','subsector_id'

**train,test**<br>
1. Convert 'first_active_month' column to "first_month","first_year","first_season"
+ Drop 'first_active_month'
+ Save to train_m.csv

In [51]:
test.loc[:,'first_active_month'].apply(type)

0         <class 'str'>
1         <class 'str'>
2         <class 'str'>
3         <class 'str'>
4         <class 'str'>
5         <class 'str'>
6         <class 'str'>
7         <class 'str'>
8         <class 'str'>
9         <class 'str'>
10        <class 'str'>
11        <class 'str'>
12        <class 'str'>
13        <class 'str'>
14        <class 'str'>
15        <class 'str'>
16        <class 'str'>
17        <class 'str'>
18        <class 'str'>
19        <class 'str'>
20        <class 'str'>
21        <class 'str'>
22        <class 'str'>
23        <class 'str'>
24        <class 'str'>
25        <class 'str'>
26        <class 'str'>
27        <class 'str'>
28        <class 'str'>
29        <class 'str'>
              ...      
123593    <class 'str'>
123594    <class 'str'>
123595    <class 'str'>
123596    <class 'str'>
123597    <class 'str'>
123598    <class 'str'>
123599    <class 'str'>
123600    <class 'str'>
123601    <class 'str'>
123602    <class 'str'>
123603    <class

In [53]:
train["first_month"] = train.loc[:,'first_active_month'].apply(lambda x: int(x.split("-")[1]))

In [59]:
test["first_month"] = test.loc[:,'first_active_month'].to_string().apply(lambda x: int(x.split("-")[1]))

AttributeError: 'str' object has no attribute 'apply'

In [20]:
train["first_year"] = train.loc[:,'first_active_month'].apply(lambda x: int(x.split("-")[0]))

In [26]:
def ssn(x):
    x = int(x.split("-")[1])
    if x in [12,1,2]:
        return 0
    elif x<6:
        return 1
    elif x<9:
        return 2
    else:
        return 3

In [27]:
train["first_season"] = train.loc[:,'first_active_month'].apply(ssn)

In [43]:
train = train.drop(['first_active_month'],axis = 1)

In [47]:
train.to_csv(DATA_PATH+r"train_m.scv")

Discard rows with NaN
Change "authorized_flag" 'Y' - > 10, 'N' - >1
Change "category_1"  'Y' - > 1, 'N' - >0
Change "category_3"  "A","B","C" - > 1,2,3
Change "purchase_date" to numerical
SUM log base = "city_id" (authorized_flag*category_1*installments*category_3*month_lag*purchase_amount*purchase_date*category_2*state_id*subsector_id)^merchant_category_id

In [41]:
hist.loc[:,"subsector_id"].unique()

array([37, 16, 34, 41, 29,  7, 33,  2, 36, 25, 15, 19, 10, 30, 18,  8, 27,
        1, 39, 32, 22, 20, 21,  9, 17, 38, 12, 26, 31, 13, 35,  5, 23,  3,
        4, 14, 24, 40, 11, -1, 28], dtype=int64)

In [29]:
new_merchant.head(10)

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_415bb3a509,107,N,1,B,307,M_ID_b0c793002c,1,-0.557574,2018-03-11 14:57:36,1.0,9,19
1,Y,C_ID_415bb3a509,140,N,1,B,307,M_ID_88920c89e8,1,-0.569580,2018-03-19 18:53:37,1.0,9,19
2,Y,C_ID_415bb3a509,330,N,1,B,507,M_ID_ad5237ef6b,2,-0.551037,2018-04-26 14:08:44,1.0,9,14
3,Y,C_ID_415bb3a509,-1,Y,1,B,661,M_ID_9e84cda3b1,1,-0.671925,2018-03-07 09:43:21,NaN,-1,8
4,Y,C_ID_ef55cf8d4b,-1,Y,1,B,166,M_ID_3c86fa3831,1,-0.659904,2018-03-22 21:07:53,NaN,-1,29
5,Y,C_ID_ef55cf8d4b,231,N,1,B,367,M_ID_8874615e00,2,-0.633007,2018-04-02 12:53:28,1.0,9,16
6,Y,C_ID_ef55cf8d4b,69,N,1,B,333,M_ID_6d061b5ddc,1,5.263697,2018-03-28 19:50:19,1.0,9,10
7,Y,C_ID_ef55cf8d4b,231,N,1,B,307,M_ID_df1e022f41,2,-0.553787,2018-04-05 08:06:52,1.0,9,19
8,Y,C_ID_ef55cf8d4b,69,N,1,B,278,M_ID_d15eae0468,2,-0.596643,2018-04-07 18:37:40,1.0,9,37
9,Y,C_ID_ef55cf8d4b,69,N,1,B,437,M_ID_5f9bffd028,1,-0.607191,2018-03-17 18:10:41,1.0,9,15


In [23]:
test_m.shape

(11082352, 18)

In [28]:
train.loc[:,"first_active_month"] = train.loc[:,"first_active_month"].apply(lambda x: (pd.to_datetime(x,format = "%Y-%m")-pd.to_datetime("1970-01-01")).days)

In [29]:
test.loc[:,"first_active_month"] = test.loc[:,"first_active_month"].apply(lambda x: (pd.to_datetime(x,format = "%Y-%m")-pd.to_datetime("1970-01-01")).days)

In [30]:
train_m = pd.merge( train, hist, how = "inner", on = "card_id")
#(18030009, 19)
#'first_active_month', 'card_id', 'feature_1', 'feature_2', 'feature_3', 'target'
#'authorized_flag', 'card_id', 'city_id', 'category_1', 'installments',
#'category_3', 'merchant_category_id', 'merchant_id', 'month_lag',
#'purchase_amount', 'purchase_date', 'category_2', 'state_id','subsector_id'

In [31]:
test_m = pd.merge( test, hist, how = "inner", on = "card_id")
#(11082352, 18)
#'first_active_month', 'card_id', 'feature_1', 'feature_2', 'feature_3',
#'authorized_flag', 'card_id', 'city_id', 'category_1', 'installments',
#'category_3', 'merchant_category_id', 'merchant_id', 'month_lag',
#'purchase_amount', 'purchase_date', 'category_2', 'state_id','subsector_id'

In [24]:
train_m.to_csv(DATA_PATH+r"train_m.csv")

In [26]:
test_m.to_csv(DATA_PATH+r"test_m.csv")

KeyboardInterrupt: 

In [ ]:
part = np.random.rand(len(df_train))<0.7

In [ ]:
valid_m = df_train[~part]